# Automatic Inference in Pyro
- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/ppl-pyro-intro.png

## Introduction

In the previous post we introduced pyro and its building blocks such as schotastic function, primitive sample and param primitive statement, model and guide. We also defined pyro model and use it to generate data, learn from data and predict future observations.

In this section, we will learn in details about inference in Pyro, how to use Pyro primitives and the effect handling library (pyro.poutine) to build custom tools for analysis.

In [2]:
import torch
import pyro
import pyro.distributions as dist
from torch.distributions import constraints
%matplotlib inline


Consider a previous poison regression model

In [8]:
def model_global_trend(p):
    slope = pyro.param('slope', torch.tensor(0.))
    sigma = pyro.param('sigma', torch.tensor(1.0), constraints=dist.constraints.positive)
    intercept = pyro.param('intercept', torch.tensor(13.))
    for i in range(len(y)):
        log_p_hat = slope * i + intercept
        pyro.sample('y_{}'.format(i), dist.LogNormal(log_p_hat, sigma), obs=p[i])

In [ ]:
From the given  model above , ```pyro.param``` designate model parameters that we would like to optimize. Observations are denoted by the obs= keyword argument to pyro.sample. This specifies the likelihood function. Instead of log transforming the data, we use a LogNormal distribution. The observations are conditionally independent given the latent random variable slope and intercept. To explicitly mark this in Pyro, **plate** statement is used to construct conditionally independent sequences of variables.

```python
with pyro.plate("name", size, subsample_size, device) as ind:
    # ...do conditionally independent stuff with ind...
```
However compared to ``range()`` each invocation of **plate** requires the user to provide a unique name. The **plate**  statement can be used either sequentially as a generator or in parallel as a context manager. Sequential plate is similar to ``range()``in that it generates a sequence of values. 
```python
 # This version declares sequential independence and subsamples data:
    for i in plate('data', 100, subsample_size=10):
         if z[i]:  # Control flow in this example prevents vectorization.
                obs = sample('obs_{}'.format(i), dist.Normal(loc, scale), obs=data[i])
```
Vectorized plate is similar to ``torch.arange()`` in that it yields an array of indices by which other tensors can be indexed. However, unlike  ``torch.arange()`` **plate**  also informs inference algorithms that the variables being indexed are conditionally independent.
```python
     # This version declares vectorized independence:
     with plate('data'):
            obs = sample('obs', dist.Normal(loc, scale), obs=data)
```
Additionally, plate can take advantage of the conditional independence assumptions by subsampling the indices and informing inference algorithms to scale various computed values. This is typically used to subsample minibatches of data:
```python
with plate("data", len(data), subsample_size=100) as ind:
    batch = data[ind]
    assert len(batch) == 100
```